# Debug detsim city NB

### JAH 12/12/19

In [ ]:
import time

%load_ext autoreload
%autoreload 2

import numpy             as np
import scipy             as sc
import scipy.stats       as st
import tables            as tb

from typing    import Callable
from typing    import Tuple
from typing    import List

import invisible_cities.io.mcinfo_io               as mcio
import invisible_cities.core    .system_of_units_c as system_of_units
import invisible_cities.core    .fit_functions     as fitf
import invisible_cities.database.load_db           as db

import myhistos                  as ht
import detsim.simulation.detsim  as ds

units = system_of_units.SystemOfUnits()

# Plotting configuration

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
np.warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"]          = 5, 4
plt.rcParams["font.size"]               = 12
plt.rcParams["figure.max_open_warning"] = 100

## DetSim

In [ ]:
detector, run_number = 'new', -1
detsimparams = ds.DetSimParameters(detector, run_number)
#detsimparams.next_detector(db)
print(detsimparams.nsipms)

In [ ]:
plt.scatter(detsimparams.x_sipms, detsimparams.y_sipms, s = 1.)
plt.xlabel('x (mm)'); plt.ylabel('y (mm)');
plt.figure()
plt.scatter(detsimparams.x_pmts, detsimparams.y_pmts, s = 1e2)
plt.xlabel('x (mm)'); plt.ylabel('y (mm)');

In [ ]:
dsim = detsimparams
npmts_bins  = int(dsim.wf_buffer_time // dsim.wf_pmt_bin_time)
npmts       = dsim.npmts

nsipms_bins = int(dsim.wf_buffer_time // dsim.wf_sipm_bin_time)
nsipms      = dsim.nsipms

drift_velocity    = dsim.drift_velocity

adc_to_pes_pmts   = dsim.adc_to_pes_pmts
adc_to_pes_sipms  = dsim.adc_to_pes_sipms

## Data

In [ ]:
datadir      = '/Users/hernando/investigacion/NEXT/work/detsim/detsim/test_data/'
datafilename = 'neut_full_test.sim.h5'

In [ ]:
def hits_generator():
    datahits = mcio.load_mchits_df(datadir + datafilename)
    hitsgroup = datahits.groupby('event_id')
    print('size ', len(hitsgroup))
    for ievt, hitsdf in hitsgroup:
        print('event ', ievt)
        yield(ievt, hitsdf)

In [ ]:
it = hits_generator()

In [ ]:
evt, hits = next(it)

In [ ]:
xs, ys, zs, enes = ds.get_deposits(hits)
print('total energy', np.sum(enes) / units.keV, ' keV');

In [ ]:
ht.graph_event(xs, ys, zs, enes, scale = 1e4)

## Generate - drift - diffuse electrons

In [ ]:
nes                 = ds.generate_electrons(enes)
nes                 = ds.drift_electrons(zs, nes)
dxs, dys, dzs, dnes = ds.diffuse_electrons(xs, ys, zs, nes)

In [ ]:
plot = True
# generate deposits
#xs, ys, zs, enes = ds.generate_deposits(xsigma = 0.4)
if (plot):
    print('total energy ', np.sum(enes) / units.keV, ' keV')
    ht.hist(xs); plt.xlabel('deposits x (mm)')
    ht.hist(ys); plt.xlabel('deposits y (mm)')
    ht.hist(zs); plt.xlabel('deposits z (mm)')
    ht.hist(enes / units.eV); plt.xlabel('deposits energy (eV)')
    
if (plot):
    print('number of secondary elctrons ', np.sum(nes))
    ht.hist(nes, 100); plt.xlabel('number of electrons')

if (plot):
    print('number of drifted electrons ', np.sum(nes));
    ht.hist(nes, 100); plt.xlabel('number of drifted electrons')

# diffuse electrons
dts                 = dzs / detsimparams.drift_velocity
if (plot):
    print('number of diffused electrons ', np.sum(nes));
    print('longitudinal diffusion ', detsimparams.longitudinal_diffusion * np.sqrt(np.mean(zs)))
    print('transverse diffusion'   , detsimparams.transverse_diffusion * np.sqrt(np.mean(zs)))
    ht.hist(dxs  , 100); plt.xlabel('diffused electrons x (mm)')
    ht.hist(dys  , 100); plt.xlabel('diffused electrons y (mm)')
    ht.hist(dzs  , 100); plt.xlabel('diffused electrons z (mm)')
    ht.hist(dts  , 100); plt.xlabel('diffused electrons time')
    ht.hist(dnes , 100); plt.xlabel('diffused electrons')

In [ ]:
#ht.graph_event( xs,  ys,  zs, enes                  , scale = 1e4);
ht.graph_event(dxs, dys, dzs, dnes * detsimparams.wi, scale = 1e4);

In [ ]:
dts                 = dzs / drift_velocity
s2photons           = ds.generate_s2_photons(dnes)
pes_pmts            = ds.estimate_s2_pes_at_pmts (dxs, dys, s2photons)
pes_sipms           = ds.estimate_s2_pes_at_sipms(dxs, dys, s2photons)

In [ ]:
def get_xybins():
    def _bins(xxs):
        xs = np.sort(xxs)
        dx   = 0.5*(xs[1] - xs[0])
        bins = np.append(xs - dx, xs[-1] + dx)
        return bins
    return _bins(detsimparams.x_sipms), _bins(detsimparams.y_sipms)   

if (plot):
    print('number of EL photons', np.sum(s2photons))
    ht.hist(s2photons, 100); plt.xlabel('photons per electron');

if (plot):
    print('pes ', np.sum(pes_pmts, axis = 0))
    ht.hist(pes_pmts, 100); plt.xlabel('pes at pmts')
    
if (plot):
    print('pes ', np.sum(pes_sipms, axis = 0))
    ht.hist(pes_sipms, 100); plt.xlabel('pes at pmts')
    plt.figure();
    xybins = get_xybins()
    detsimparams.x_sipms[1] - detsimparams.x_sipms[0]
    int_pes_sipms  = np.sum(pes_sipms, axis = 0)
    plt.hist2d(detsimparams.x_sipms, detsimparams.y_sipms, bins = xybins, weights = int_pes_sipms);
    plt.figure()
    plt.hist2d(detsimparams.x_sipms, detsimparams.y_sipms, bins = xybins, weights = int_pes_sipms);
    plt.xlim((np.min(dxs) - 20, np.max(dxs) + 20))
    plt.ylim((np.min(dys) - 20, np.max(dys) + 20))

In [ ]:
s1photons           = ds.generate_s1_photons(enes)
pes_s1              = ds.estimate_s1_pes(xs, ys, zs, s1photons)

In [ ]:
print('S1 photons ', np.sum(s1photons))
if (plot):
    print('pes ', np.sum(pes_s1, axis = 0))
    ht.hist(pes_s1, 100); plt.xlabel('pes S1')

In [ ]:
t0                  = ds.find_trigger_time(dts, pes_pmts)
t0s                 = t0 * np.ones_like(zs)
dts                 = t0 + dts

In [ ]:
print('trigger time ', t0 / units.mus, np.min(dts)/ units.mus)

In [ ]:
wfs_pmts            = np.zeros((npmts_bins, npmts), dtype = int)
ds.fill_wfs_s1     (t0s, pes_s1  , wfs_pmts)
ds.fill_wfs_s2_pmts(dts, pes_pmts, wfs_pmts);

In [ ]:
times_pmts   = np.arange(npmts_bins) * detsimparams.wf_pmt_bin_time
#print(times_pmts.shape, wfs_pmts.shape)
if (plot):
    xcenters = times_pmts
    plt.plot(xcenters / units.mus, wfs_pmts); plt.xlabel('wfs pmts')
    plt.figure()
    plt.plot(xcenters / units.mus, wfs_pmts); plt.xlabel('wfs pmts')
    plt.xlim((np.min(dts) /units.mus - 5, np.max(dts) / units.mus + 10));

In [ ]:
wfs_sipms           = np.zeros((nsipms_bins, nsipms), dtype = int)
ds.fill_wfs_s2_sipms(dts, pes_sipms, wfs_sipms);

In [ ]:
times_sipms   = np.arange(nsipms_bins) * detsimparams.wf_sipm_bin_time
if (plot):
    xcenters = times_sipms
    plt.plot(xcenters / units.mus, wfs_sipms); plt.xlabel('wfs sipms')
    plt.xlim((np.min(dts) /units.mus - 5, np.max(dts) / units.mus + 10));

In [ ]:
wfs_pmts            = wfs_pmts  * adc_to_pes_pmts
sipms_pmts          = wfs_sipms * adc_to_pes_sipms

In [ ]:
times_pmts   = np.arange(npmts_bins) * detsimparams.wf_pmt_bin_time
#print(times_pmts.shape, wfs_pmts.shape)
if (plot):
    xcenters = times_pmts
    plt.plot(xcenters / units.mus, wfs_pmts); plt.xlabel('wfs pmts')
    plt.figure()
    plt.plot(xcenters / units.mus, wfs_pmts); plt.xlabel('wfs pmts')
    plt.xlim((np.min(dts) /units.mus - 5, np.max(dts) / units.mus + 10));

In [ ]:
times_sipms   = np.arange(nsipms_bins) * detsimparams.wf_sipm_bin_time
if (plot):
    xcenters = times_sipms
    plt.plot(xcenters / units.mus, wfs_sipms); plt.xlabel('wfs sipms')
    plt.xlim((np.min(dts) /units.mus - 5, np.max(dts) / units.mus + 10));

In [ ]:
## One go

In [ ]:
def plot_wfs(wfs_pmts, wfs_sipms):
    nbins  = int(detsimparams.wf_buffer_time / detsimparams.wf_pmt_bin_time)
    times  = np.arange(nbins) * detsimparams.wf_pmt_bin_time
    plt.figure()
    plt.plot(times / units.mus, wfs_pmts); plt.xlabel('wfs pmts')
    plt.figure()
    plt.plot(times / units.mus, wfs_pmts); plt.xlabel('wfs pmts')
    plt.xlim((np.min(dts) /units.mus - 5, np.max(dts) / units.mus + 10));

    nbins  = int(detsimparams.wf_buffer_time / detsimparams.wf_sipm_bin_time)
    times  = np.arange(nbins) * detsimparams.wf_sipm_bin_time
    plt.figure()
    plt.plot(times / units.mus, wfs_sipms); plt.xlabel('wfs sipms')
    plt.figure()
    plt.plot(times / units.mus, wfs_sipms); plt.xlabel('wfs sipms')
    plt.xlim((np.min(dts) /units.mus - 5, np.max(dts) / units.mus + 10));

In [ ]:
generate_wfs = ds.get_function_generate_wfs()
wfs_pmts, wfs_sipms = generate_wfs(hits)
if (plot): plot_wfs(wfs_pmts, wfs_sipms)

## Timings

In [ ]:
def time_to_simulate_one_event(hits, n = 1):
    t0 = time.time()
    for i in range(n):
        generate_wfs(hits)
    return (time.time() - t0) / n

In [ ]:
dtsim = time_to_simulate_one_event(hits, n = 10)
print('total ', dtsim, 's')

In [ ]:
evt, hits = next(it)
xs, ys, zs, enes = ds.get_deposits(hits)
print('total energy', np.sum(enes) / units.keV, ' keV');
ht.graph_event(xs, ys, zs, enes, scale = 1e4)
dtsim = time_to_simulate_one_event(hits, n = 10)
print('total ', dtsim, 's')

In [ ]:
evt, hits = next(it)
xs, ys, zs, enes = ds.get_deposits(hits)
print('total energy', np.sum(enes) / units.keV, ' keV');
ht.graph_event(xs, ys, zs, enes, scale = 1e4)
dtsim = time_to_simulate_one_event(hits, n = 10)
print('total ', dtsim, 's')